In [1]:
!pip install gensim

In [21]:
import pandas as pd
import numpy as np
import ssl
import gensim
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ssl._create_default_https_context = ssl._create_unverified_context


class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.loss_previous_step = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss
        

class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        print(f'Epoch {self.epoch}')
        self.epoch += 1

        

# TF-IDF и Suggest

Этот семинар будет посвящен работе с текстами. Мы будем решать задачу о которой говорили на лекции: будем реализовывать suggest.

### План семинара


* [Задача suggest: многоклассовая классификация](#Задача-suggest)
* [Обучаем модель линейной регрессии](#Обучаем-модель-лог-регрессии)
    - Делаем бейзлайн
    - Улучшаем бейзлайн
    
* Обучаем word2vec
* Вычисляем признаки заголовков
* Смотрим метрику в нашей задаче

**Работа на семинаре**
* Реализуем tf-idf 

**Домашнее задание**
* Реализовать класс сервиса suggest-by-title

### Задача suggest

Подача без suggest:

<img src="https://ucarecdn.com/05ab935a-ff65-4d75-afcb-6eb0d71c5d44/" width="700">

Подача с suggest:
<img src="https://ucarecdn.com/5e1684f1-eec5-4054-9e43-08be8d9acbcb/" width="700">

Задача заключается в том, чтобы определить к какой категории относится объявление используя заголовок. Категорий в выборке 54, задача, соответственно многоклассовая классификация.

Задача многоклассовой классификации методом one-versus-all:

<img src="https://ucarecdn.com/2abe5812-3f6e-42cf-97a4-a35a6de58c6d/" width="534">

### Смотрим на данные

In [3]:
data_train = pd.read_csv('suggest_train.csv', index_col=0)
data_test = pd.read_csv('suggest_test.csv', index_col=0)
w2v_train = pd.read_csv('unlabeled_data.csv', index_col=0)

In [4]:
data_train.head()

,title,category_id,name
item_id,,,
335330,Раствор. Товарный бетон от производителя,15,Для дома и дачи|Ремонт и строительство|Стройма...
137480,Детская кроватка,16,"Для дома и дачи|Мебель и интерьер|Кровати, див..."
242690,"Кровать -качалка с ящиком для вещей, два полож...",34,Личные вещи|Товары для детей и игрушки|Детская...
434016,Линолеум комитекс лин печора орфей 431,15,Для дома и дачи|Ремонт и строительство|Стройма...
237890,Распродажа имущества предприятия. Компьютеры,7,Бытовая электроника|Настольные компьютеры


### Обучаем модель линейной регрессии

#### Разбиваем данные на трейн и валидацию

Идеи:
- Частые слова зашумляют выборку;
- Редкие слова дают модели переобучиться;
- n-граммы позволят частично учитывать порядок;
- Регуляризация позволит уменьшить эффект переобучения;
- ???

In [123]:
train_titles, val_titles, y_train, y_val = train_test_split(
    data_train['title'],
    data_train['category_id'],
    random_state=42
)

In [6]:
count_vectorizer = CountVectorizer()
count_vectorizer

CountVectorizer()

In [7]:
X_train = count_vectorizer.fit_transform(train_titles.values)
X_val = count_vectorizer.transform(val_titles.values)

In [8]:
print(len(count_vectorizer.vocabulary_))
count_vectorizer.vocabulary_

17203


{'бильярдный': 7912,
 'стол': 15273,
 'крассула': 10951,
 'оватта': 12566,
 'денежное': 9243,
 'дерево': 9257,
 'фиксатор': 16165,
 'вальгус': 8240,
 'про': 13737,
 'подставка': 13385,
 'для': 9403,
 'цветов': 16486,
 'комплект': 10695,
 'мебели': 11705,
 'ибица': 9941,
 'wi': 6974,
 'fi': 3596,
 'роутер': 14329,
 'link': 4611,
 'dir': 3143,
 '615': 1598,
 'электрический': 16976,
 'самовар': 14492,
 'sony': 6238,
 'experia': 3500,
 'snell': 6208,
 'farcry': 3557,
 'ps4': 5635,
 'продам': 13770,
 'смартфон': 14896,
 'nokia': 5241,
 'lumia': 4694,
 '630': 1613,
 'dual': 3234,
 'чёрный': 16667,
 'minelab': 4954,
 '17': 424,
 'smart': 6186,
 'ctx': 2971,
 '3030': 1017,
 'ведьмак': 8295,
 'дикая': 9354,
 'охота': 12824,
 'продаются': 13774,
 'коньки': 10764,
 'фигурные': 16158,
 'женские': 9672,
 'ретро': 14217,
 'свеча': 14601,
 'очень': 12842,
 'красивая': 10928,
 'настенная': 12259,
 'полка': 13463,
 'навесная': 12162,
 'пиджак': 13124,
 'из': 9982,
 'экокожи': 16935,
 'со': 14944,
 'сту

In [9]:
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs')
log_reg.fit(X_train, y_train)

LogisticRegression()

In [10]:
preds = log_reg.predict(X_val)
accuracy_score(preds, y_val)

0.739

### Учим w2v

Идеи:
- Обучить качественные векторные представление слов на неразмеченных данных;
- Получить из них вектора заголовков;
- Оценить адекватность;
- Оценить качество решения нашей задачи;
- Большая размерность позволит получить более качественные векторные представления;
- Сравнить CBOW и skip-gram;
- ???

In [22]:
import re
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

WORD_PATTERN = '(?u)\\b\\w\\w+\\b'

reg_exp = re.compile(pattern=WORD_PATTERN)
w2v_train_data = pd.read_csv('unlabeled_data.csv',  index_col=0)
print(w2v_train_data.shape)
w2v_train_data.head()

(200000, 2)


,item_id,title
30000,377939,1 рубль царские 10 штук
30001,384779,"Напольный насос giyo высокого давления, с мано..."
30002,276889,Sony Xperia m2 Dual
30003,155718,Zoom B2.1U+ AC Процессор эффектов для бас-гитары
30004,260705,Люстра Favourite petite 1574-7P


In [13]:
sentences = [reg_exp.findall(s.lower()) for s in w2v_train_data.title]
sentences[:5]

[['рубль', 'царские', '10', 'штук'],
 ['напольный', 'насос', 'giyo', 'высокого', 'давления', 'манометр'],
 ['sony', 'xperia', 'm2', 'dual'],
 ['zoom', 'b2', '1u', 'ac', 'процессор', 'эффектов', 'для', 'бас', 'гитары'],
 ['люстра', 'favourite', 'petite', '1574', '7p']]

In [108]:
w2v_model = Word2Vec(sg=1, vector_size=200, window=5, min_count=3, hs=1, negative=12)
w2v_model.build_vocab(sentences)
w2v_model.train(
    sentences,
    total_examples=w2v_model.corpus_count,
    epochs=50,
    compute_loss=True,
    callbacks=[LossLogger()]
)

Loss after epoch 0: 3059256.25
Loss after epoch 1: 2194016.75
Loss after epoch 2: 1941359.0
Loss after epoch 3: 1781419.0
Loss after epoch 4: 1579477.0
Loss after epoch 5: 1531855.0
Loss after epoch 6: 1522879.0
Loss after epoch 7: 1497039.0
Loss after epoch 8: 1476703.0
Loss after epoch 9: 1097924.0
Loss after epoch 10: 1033648.0
Loss after epoch 11: 1015964.0
Loss after epoch 12: 1015168.0
Loss after epoch 13: 1001242.0
Loss after epoch 14: 992018.0
Loss after epoch 15: 986758.0
Loss after epoch 16: 987446.0
Loss after epoch 17: 973238.0
Loss after epoch 18: 969504.0
Loss after epoch 19: 959824.0
Loss after epoch 20: 954220.0
Loss after epoch 21: 953956.0
Loss after epoch 22: 942324.0
Loss after epoch 23: 935746.0
Loss after epoch 24: 933738.0
Loss after epoch 25: 928354.0
Loss after epoch 26: 514960.0
Loss after epoch 27: 325740.0
Loss after epoch 28: 322000.0
Loss after epoch 29: 317464.0
Loss after epoch 30: 313628.0
Loss after epoch 31: 308852.0
Loss after epoch 32: 297292.0
Loss

(26655014, 36711850)

In [117]:
fasttext = FastText(sg=1, vector_size=150, window=3, min_count=3, hs=1, negative=5)
fasttext.build_vocab(sentences)
fasttext.train(
    sentences,
    total_examples=fasttext.corpus_count,
    epochs=50,
    compute_loss=True,
    callbacks=[LossLogger()]
)

Loss after epoch 0: 0.0
Loss after epoch 1: 0.0
Loss after epoch 2: 0.0
Loss after epoch 3: 0.0
Loss after epoch 4: 0.0
Loss after epoch 5: 0.0
Loss after epoch 6: 0.0
Loss after epoch 7: 0.0
Loss after epoch 8: 0.0
Loss after epoch 9: 0.0
Loss after epoch 10: 0.0
Loss after epoch 11: 0.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0
Loss after epoch 20: 0.0
Loss after epoch 21: 0.0
Loss after epoch 22: 0.0
Loss after epoch 23: 0.0
Loss after epoch 24: 0.0
Loss after epoch 25: 0.0
Loss after epoch 26: 0.0
Loss after epoch 27: 0.0
Loss after epoch 28: 0.0
Loss after epoch 29: 0.0
Loss after epoch 30: 0.0
Loss after epoch 31: 0.0
Loss after epoch 32: 0.0
Loss after epoch 33: 0.0
Loss after epoch 34: 0.0
Loss after epoch 35: 0.0
Loss after epoch 36: 0.0
Loss after epoch 37: 0.0
Loss after epoch 38: 0.0
Loss after epoch 39: 0.0
Loss after

(30686138, 36711850)

In [71]:
w2v_model.wv.similar_by_word('продам')

[('продаю', 0.9604673385620117),
 ('породам', 0.7619137763977051),
 ('новую', 0.6197563409805298),
 ('продаются', 0.6033662557601929),
 ('продадим', 0.5792599320411682),
 ('хорошую', 0.5781928896903992),
 ('обменяю', 0.5660350918769836),
 ('срочно', 0.5624358654022217),
 ('подам', 0.5613104701042175),
 ('прдаю', 0.5541633367538452)]

In [72]:
w2v_model.wv.similar_by_word('айфон')

[('iphone', 0.7975996732711792),
 ('4s', 0.7229064702987671),
 ('5s', 0.7223721742630005),
 ('iphone5', 0.7081241011619568),
 ('селикон', 0.7000847458839417),
 ('5с', 0.6986843943595886),
 ('ipone', 0.6905215978622437),
 ('айфоны', 0.6752544641494751),
 ('айфон4', 0.6624860763549805),
 ('5c', 0.6389901041984558)]

In [149]:
class Word2VecTransformer:
    
    def __init__(self, w2v_model, word_pattern):
        
        self.w2v_model = w2v_model
        self.word_pattern = word_pattern
        self.re = re.compile(pattern=self.word_pattern)
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        
        X_transformed = np.zeros((len(X), self.w2v_model.wv.vector_size))
        for i, title in enumerate(X):
            
            title_vector = np.zeros((self.w2v_model.wv.vector_size,))
            tokens = self.re.findall(title.lower())
            for token in tokens:
                if token in self.w2v_model.wv.key_to_index:
                    title_vector += self.w2v_model.wv.get_vector(token)
                    
            X_transformed[i] = title_vector / (1 if len(tokens) == 0 else len(tokens))
        
        return X_transformed

In [151]:
w2v_transformer = Word2VecTransformer(w2v_model=w2v_model, word_pattern=WORD_PATTERN)
ft_transformer = Word2VecTransformer(w2v_model=fasttext, word_pattern=WORD_PATTERN)

train_w2v = w2v_transformer.transform(train_titles.values)
val_w2v = w2v_transformer.transform(val_titles.values)
train_ft = ft_transformer.transform(train_titles.values)
val_ft = ft_transformer.transform(val_titles.values)
full_ft = ft_transformer.transform(data_train['title'].values)

In [129]:
log_reg = LogisticRegression(solver='lbfgs', max_iter=1000)
log_reg.fit(full_ft, data_train['category_id'].values)

LogisticRegression(max_iter=1000)

In [58]:
def accuracy_score_3(preds, targets):
    size = len(targets)
    s = 0
    for idx, pred in enumerate(preds):
        if targets[idx] in pred:
            s += 1
    return s / size

In [131]:
predict_top3 = np.argsort(-log_reg.predict_proba(full_ft), axis=-1)[:, :3]
accuracy_score_3(predict_top3, data_train['category_id'].values)

0.93955

In [153]:
test_titles = np.array(data_test.index)
test_ft = ft_transformer.transform(test_titles)
predict = np.argsort(-log_reg.predict_proba(test_ft), axis=-1)[:, :3]
pd.DataFrame(predict).to_csv('solution.csv', header=['top1', 'top2', 'top3'], index=False)

### Домашнее задание

Теоритическая часть (обязательно).

1. Посмотреть видео на [3BlueBrowb](https://youtu.be/aircAruvnKk)

Практическая часть.

**Реализовать класс Suggester, который возвращает от 1 до 5 наиболее вероятных категорий по введённой строке.**

accuracy по top1 > 0.78

In [ ]:
from typing import List

class Suggester:
    
    def __init__(self, max_suggest_count=5, default_suggest=''):
        self.max_suggest_count = max_suggest_count
        self.default_suggest = default_suggest
        
    def suggest(self, title: str):
        print('Для дома и дачи|Посуда и товары для кухни|Посуда')
        print('Хобби и отдых|Коллекционирование|Другое')
    
    def test_suggest(self):
        
        title = input()
        while title != 'stop':
            
            suggest = self.suggest(title)

            if suggest:
                print(suggest)
            else:
                print(self.default_suggest)
            
            title = input()

In [ ]:
sug = Suggester()
sug.test_suggest()